<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/master/notebooks/RtGender-Notebooks/RtGender_Annotations_Sentiment_Easy_Data_Augmentation_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RtGender - Annotations - Sentiment w/ Easy Data Augmentation Techniques
[Source](https://github.com/jasonwei20/eda_nlp)

[Paper that explores these techniques](https://arxiv.org/abs/1901.11196)


In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = r'/content/drive/MyDrive/w266'
except ModuleNotFoundError:
  path = r'data'

Mounted at /content/drive


<a id='section01'></a>
## Load Modules

In [2]:
!pip install -U nltk
import nltk; nltk.download('wordnet')

     |████████████████████████████████| 1.5 MB 7.4 MB/s 
     |████████████████████████████████| 748 kB 46.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
%%capture
#!pip install transformers==3.0.2
!pip install -q transformers

In [4]:
%%capture
!pip install datasets
import datasets 
from datasets import load_dataset, Dataset, DatasetDict

In [5]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn import metrics
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import logging
logging.basicConfig(level=logging.ERROR)

In [6]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
## Import and Reshape Data

In [7]:
train_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_dev.csv')

print('train_shape: ',train_df.shape)
print('dev_shape: ',dev_df.shape)

train_shape:  (10746, 8)
dev_shape:  (2303, 8)


In [8]:
# there are NaNs in the dev dataset remove 
nan_values = dev_df[dev_df.isna().any(axis=1)] 
print(nan_values)

# return without missing values in response_text
dev_df.dropna(subset = ["response_text"], inplace=True)

print("Train shape", train_df.shape)
print("Dev shape", dev_df.shape)

      Unnamed: 0         source op_gender  ... sentiment   relevance label
830         2576  facebook_wiki         M  ...   Neutral  Irrelevant     1
1664        2722  facebook_wiki         W  ...   Neutral  Irrelevant     1

[2 rows x 8 columns]
Train shape (10746, 8)
Dev shape (2301, 8)


In [9]:
print("Unique sentiments: ", train_df['sentiment'].unique())

Unique sentiments:  ['Positive' 'Mixed' 'Neutral' 'Negative']


In [10]:
# Change Mixed to Neutral
train_df['sentiment'].replace({"Mixed":"Neutral"}, inplace = True)
dev_df['sentiment'].replace({"Mixed":"Neutral"}, inplace = True)

# Run through Easy Data Augmentation Techniques package
[augment](https://github.com/jasonwei20/eda_nlp/blob/master/code/augment.py)


#### Import Modules

In [11]:
# import eda script from github
!git clone https://github.com/jasonwei20/eda_nlp

Cloning into 'eda_nlp'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 396 (delta 6), reused 8 (delta 4), pack-reused 379
Receiving objects: 100% (396/396), 20.42 MiB | 21.84 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [12]:
import sys
# sys.path is a list of absolute path strings
sys.path.append('/content/eda_nlp/code/')

from eda import *


#### Random deletion
Randomly delete words from the sentence with probability p


In [13]:
# apply the main data augmentation function in the eda module

augmented_sentences = []

for sentence in train_df['response_text']:
  words = sentence.split(' ')
  a_words = random_deletion(words, 0.1)
  augmented_sentences.append(' '.join(a_words))


#### Replace with Synonyms

In [14]:
augmented_sentences_2 = []
alpha_sr=0.1

if (alpha_sr > 0):
  for sentence in augmented_sentences:
    words = sentence.split(' ')
    num_words = len(sentence)
    n_sr = max(1, int(alpha_sr*num_words))
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences_2.append(' '.join(a_words))

In [15]:
# add to train df
train_df_aug = train_df
train_df_aug['augmented_response_text'] = augmented_sentences_2
train_df_aug

,Unnamed: 0,source,op_gender,post_text,response_text,sentiment,relevance,label,augmented_response_text
0,3845,facebook_congress,W,Im reading the 3/1 GAO report that finds billi...,Thank you Congresswoman Bass. Keep up the grea...,Positive,ContentPoster,2,give thanks you congresswoman Bass. proceed up...
1,9743,fitocracy,M,Being followed by the famous DBJ? Quite an honor.,"Well, I am very honored you feel so honored",Positive,Content,2,one am very you tactile property so honored
2,13041,ted,W,"Penelope Boston gave a talk about Planets, exp...",Her opinions seem driven by wishful thinking. ...,Neutral,Content,1,Her push back by desirous thinking. She there ...
3,4265,facebook_congress,W,Congress must act to help the 41 million Ameri...,There's no other way out of the enormity excep...,Positive,Content,2,There's no other way of life out of the outrag...
4,13145,ted,W,"Pardis Sabeti gave a talk about Africa, big pr...",What were the benefits of the larger community...,Neutral,Content,1,What were the of the orotund community? She di...
...,...,...,...,...,...,...,...,...,...
10741,14128,ted,M,"Bjarke Ingels gave a talk about architecture, ...",Brillant!! Ingels has a terrific future ahead ...,Positive,Content,2,Brillant!! Ingels has a terrifying future tens...
10742,5589,facebook_congress,W,I was honored to meet with Eliseo Medina and F...,The Democrats view this as another way to use ...,Negative,Content,0,The sentiment this as some other means to empl...
10743,10672,reddit,W,SO YOU LIKE STACKING CUPS?! DO WE HAVE A GREAT...,Is this real?? Well at least this kid will be ...,Neutral,Content,1,follow this real?? substantially to the lowest...
10744,4839,facebook_congress,M,Try this Brian Schatz FB bumper sticker - an e...,EH BRIAN WEA MY STICKA N WAT OBAMA STAY ON UM ...,Neutral,Irrelevant,1,BRIAN WEA MY STICKA nitrogen WAT OBAMA persist...


# Transform to HuggingFace friendly format

In [16]:
# change to dataset to work with Huggingface transformer & remove unused columns
columns_to_remove = ['op_gender', 'source', 'Unnamed: 0', 'relevance', 'sentiment','post_text']

from datasets import load_dataset
train_dataset = Dataset.from_pandas(train_df_aug)
dev_dataset = Dataset.from_pandas(dev_df)

train_dataset = train_dataset.remove_columns(column_names= columns_to_remove)
dev_dataset = dev_dataset.remove_columns(column_names= columns_to_remove)
dev_dataset = dev_dataset.remove_columns(column_names= '__index_level_0__')


# rename sentiment to labels
# train_dataset = train_dataset.rename_column("sentiment", "label")
# dev_dataset = dev_dataset.rename_column("sentiment", "label")

In [17]:
# combine into a DataDictionary for huggingface use
rtg_dataset = DatasetDict({
    'train': train_dataset,
    'dev': dev_dataset 
})

rtg_dataset

DatasetDict({
    train: Dataset({
        features: ['response_text', 'label', 'augmented_response_text'],
        num_rows: 10746
    })
    dev: Dataset({
        features: ['response_text', 'label'],
        num_rows: 2301
    })
})

## Tokenize

In [18]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
from transformers import AutoTokenizer
max_length = 287 #99% of training response_text are 287 or fewer characters

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length = max_length)

def tokenize(batch):
    return tokenizer(batch["response_text"], padding=True, truncation=True)

DEBUG:filelock:Attempting to acquire lock 140458947914896 on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 140458947914896 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140458947914896 on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Lock 140458947914896 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
DEBUG:filelock:Attempting to acquire lock 140458823817872 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 140458823817872 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140458823817872 on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Lock 140458823817872 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
DEBUG:filelock:Attempting to acquire lock 140458948743376 on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140458948743376 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140458948743376 on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Lock 140458948743376 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:filelock:Attempting to acquire lock 140458823164880 on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 140458823164880 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140458823164880 on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
DEBUG:filelock:Lock 140458823164880 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


In [20]:
rtg_encoded = rtg_dataset.map(tokenize, batched=True, batch_size=None)
rtg_encoded['train'].features

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'augmented_response_text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'response_text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

# Model

In [21]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device))

DEBUG:filelock:Attempting to acquire lock 140458796201936 on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
DEBUG:filelock:Lock 140458796201936 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140458796201936 on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
DEBUG:filelock:Lock 140458796201936 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or wi

In [22]:
rtg_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
rtg_encoded["dev"].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'response_text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [23]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_weighted = f1_score(labels, preds, average="weighted")
    f1_macro = f1_score(labels, preds, average = 'macro')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1_weighted, "f1_macro": f1_macro} 

In [24]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(rtg_encoded["train"]) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=False,
                                 # metric_for_best_model="f1_macro",
                                 # weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False
                                  )

In [25]:
from sklearn.metrics import classification_report

accuracy_list = []
weighted_f1_score_list = []
macro_f1_score_list = []
negative_f1_score = []
mixed_neutral_f1_score = []
positive_f1_score = []


for i in range(5):
  try:
    del trainer
    del results
    del cr
  except: pass


  trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=rtg_encoded["train"],
                  eval_dataset=rtg_encoded["dev"])
  trainer.train()
  results = trainer.evaluate()

  # append macro metrics to lists
  accuracy_list.append(results.get('eval_accuracy'))
  weighted_f1_score_list.append(results.get("eval_f1"))
  macro_f1_score_list.append(results.get("eval_f1_macro"))

  trainer.predict(rtg_encoded["dev"])
  # append the class-level F1 scores
  outputs = trainer.predict(rtg_encoded["dev"])
  predictions = outputs.predictions.argmax(1)
  labels = rtg_encoded["dev"]['label']
  cr = classification_report(labels, predictions, digits=3, output_dict=True)
  negative_f1_score.append(cr.get('0').get("f1-score"))
  mixed_neutral_f1_score.append(cr.get('1').get("f1-score"))
  positive_f1_score.append(cr.get('2').get("f1-score"))

  print(f'---------------------------Iteration {i+1} Complete---------------------------\n')

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text, augmented_response_text.
***** Running training *****
  Num examples = 10746
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2688


Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
1,0.666300,0.636372,0.732290,0.733478,0.700663
2,0.495400,0.699294,0.742721,0.739665,0.704734


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-1344
Configuration saved in results/checkpoint-1344/config.json
Model weights saved in results/checkpoint-1344/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-2688
Configuration saved in results/checkpoint-2688/config.json
Model weights saved in results/checkpoint-2688/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequence

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 2301
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 2301
  Batch size = 8
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text, augmented_response_text.
***** Running training *****
  Num examples = 10746
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2688


---------------------------Iteration 1 Complete---------------------------



...

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
1,0.068800,2.331839,0.715341,0.715785,0.681792
2,0.050400,2.266374,0.718818,0.717390,0.682872


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-1344
Configuration saved in results/checkpoint-1344/config.json
Model weights saved in results/checkpoint-1344/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-2688
Configuration saved in results/checkpoint-2688/config.json
Model weights saved in results/checkpoint-2688/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequence

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 2301
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Prediction *****
  Num examples = 2301
  Batch size = 8


---------------------------Iteration 5 Complete---------------------------



In [26]:
import statistics

print("%15s %s (%s)" % ("EDA Tech","Mean", "StDev"))

print("-"*29)
print("Macro Scores")
print("-"*29)

print(f"%15s %s (%s)" %("Accuracy",
    round(statistics.mean(accuracy_list),3),
    round(statistics.stdev(accuracy_list),3)))
print(f"%15s %5s (%s)" %("Macro F1",
    round(statistics.mean(macro_f1_score_list),3),
    round(statistics.stdev(macro_f1_score_list),3)))
print(f"%15s %5s (%s)" %("Weighted F1",
    round(statistics.mean(weighted_f1_score_list),3),
    round(statistics.stdev(weighted_f1_score_list),3)))

print("-"*29)
print("Class Scores")
print("-"*29)

print(f"%15s %s (%s)" %("Positive",
    round(statistics.mean(positive_f1_score),3),
    round(statistics.stdev(positive_f1_score),3)))
print(f"%15s %5s (%s)" %("Mixed/Neutral",
    round(statistics.mean(mixed_neutral_f1_score),3),
    round(statistics.stdev(mixed_neutral_f1_score),3)))
print(f"%15s %5s (%s)" %("Negative",
    round(statistics.mean(negative_f1_score),3),
    round(statistics.stdev(negative_f1_score),3)))

       EDA Tech Mean (StDev)
-----------------------------
Macro Scores
-----------------------------
       Accuracy 0.725 (0.011)
       Macro F1 0.688 (0.01)
    Weighted F1 0.723 (0.01)
-----------------------------
Class Scores
-----------------------------
       Positive 0.825 (0.009)
  Mixed/Neutral 0.639 (0.012)
       Negative 0.598 (0.01)


In [27]:
output_model_file = '/content/drive/MyDrive/w266/pytorch_bert_rtgender_easy_data_aug.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')

All files saved
